In [154]:
import httplib
import json

def have_numeric(str):
    for c in str:
        if (c.isnumeric()):
            return True
    return False

def split_with_first_numeric(av):
    if '100%' == av[:4]:
        av = av[4:]
    if (av[0].isnumeric() or av[1].isnumeric()):
        for i in range(len(av) - 1, -1, -1):
            if av[i].isnumeric():
                return [av[i + 1:], av[:i + 1]]
    print av
    for i, c in enumerate(av):
        if c.isnumeric():
            return [av[:i], av[i:]]

In [132]:
conn = httplib.HTTPSConnection("api.eu.apiconnect.ibmcloud.com")
headers = {
    'x-ibm-client-id': "7ddb67df-4c7b-4fa3-ab1f-1a5c3cd2dfc8",
    'x-ibm-client-secret': "pF0lJ3cV0vO4tJ7yF5gE5aX5oK6mI7nM8eR5cM0yI2cY8fD5lQ",
    'content-type': "application/json",
    'accept': "application/json",
    'accept-language': "fi"
    }


total = 10000
step = int(total/10)
#"""
products = []
for offset in range(0, total+1, 100):
    if offset % step == 0:
        print(int(offset/step), '/ 10...')
    payload = "{\"view\":{\"limit\":100,\"offset\":"+str(offset)+"}}"
    conn.request("POST", "/kesko-dev-rauta-api/qa/products/actions/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data.decode("utf-8")) 
    if 'results' not in data:
        break
    data = data['results']
    for product in data:
        products.append(product)       
#"""

(0, '/ 10...')


KeyboardInterrupt: 

In [155]:
productean_to_attrs = {}
productean_to_cat = {}
productean_to_img = {}
cat_to_producteans = {}

In [156]:
products = products_backup

In [163]:
fuck_up = 0
total = 0
for product in products:
    if 'attributes' in product and 'articleFeatures' in product['attributes']:
        if (len(product['attributes']['articleFeatures'].split('System.Collections')) > 1):
            #print "Fuck"
            continue
        atrs = product['attributes']['articleFeatures'].split('\n')
        for i in range(0, len(atrs)):
            if (atrs[i][0:1] == '- '):
                atrs[i] = atrs[i][2:]
        if not atrs or len(atrs)==0:
            continue
        #atrs = list(map(lambda atr:split_with_first_numeric(atr), atrs))
        total += len(atrs)
        for i in range(0, len(atrs)):
            while atrs[i] and len(atrs[i]) > 0 and (atrs[i][0] == ' ' or atrs[i][0] == ',' or atrs[i][0] == '-'):
                atrs[i] = atrs[i][1:]
            while atrs[i] and len(atrs[i]) > 0 and (atrs[i][-1] == ' ' or atrs[i][-1] == ',' or atrs[i][-1] == '-'):
                atrs[i] = atrs[i][:-1]
            if (len(atrs[i].split(" ")) == 2 and len(atrs[i].split(" ")[0]) > 0 and len(atrs[i].split(" ")[1]) > 0):
                atrs[i] = atrs[i].split(" ")
            elif (len(atrs[i].split(":")) == 2 and len(atrs[i].split(":")[0]) > 0 and len(atrs[i].split(":")[1]) > 0):
                atrs[i] = atrs[i].split(":")
            elif (have_numeric(atrs[i])):
                atrs[i] = split_with_first_numeric(atrs[i])
            else:
                fuck_up += 1
                atrs[i] = None
                continue
            if (atrs[i]):
                for j in range(0, 2):
                    while atrs[i][j] and len(atrs[i][j]) > 0 and (atrs[i][j][0] == ' ' or atrs[i][j][0] == ',' or atrs[i][j][0] == '-'):
                        atrs[i][j] = atrs[i][j][1:]
                    while atrs[i][j] and len(atrs[i][j]) > 0 and (atrs[i][j][-1] == ' ' or atrs[i][j][-1] == ',' or atrs[i][j][-1] == '-'):
                        atrs[i][j] = atrs[i][j][:-1]
                if (len(atrs[i][0]) > 0 and len(atrs[i][1]) > 0):
                    atrs[i] = (atrs[i][0], atrs[i][1])
                elif len(atrs[i][0]) > 0:
                    atrs[i] = (atrs[i][0], atrs[i][0])
                else:
                    atrs[i] = (atrs[i][1], atrs[i][1])
            
        atrs_ = []
        for a in atrs:
            if a:
                atrs_.append(a)
        atrs = atrs_
        this_ean = product['id']
        this_cat = product['categoryId']
        productean_to_cat[this_ean] = this_cat
        if this_cat not in cat_to_producteans:
            cat_to_producteans[this_cat] = []
        cat_to_producteans[this_cat] += [this_ean]
        if 'images' in product:
            productean_to_img[this_ean] = product['images'][0]['url']
        #if this_ean in productean_to_attrs:
            #print('collision!', this_ean)
        productean_to_attrs[this_ean] = {}
        for atr, value in atrs:
            productean_to_attrs[this_ean][atr] = value
print fuck_up, total

Ottoteho 440 W
Iskuluku 0-5000 min-1
Paino 1,9 kg
 puuvilla, twill, 370g/m²
 puuvilla, twill, 370g/m²
 puuvilla, twill, 370g/m²
 puuvillajersey, 180 g/m²
 puuvilla twill, 320 g/m²
 puuvilla, twill, 370g/m²
Pituus 50 mm
vedenimukykyluokka BIII (>10%)
ottoteho 750 W
nauhan koko 75 x 533 mm
nauhan nopeus 200–330 m/min.
Korkeus 0,2 m.
Leveys 2 m.
teho 80 W
terän oskillaatioliike 2,8 mm
kierroslukualue 15000-21000 min-1
kovettuu n. 15 minuutissa
ottoteho 1100 W
kierrosluku 8000-24000 min-1
syöttöliike 0-57 mm
värivaihtelu V2 (melko vähän)
vedenimukykyluokka BIII (>10%)
terän oskillaatioliike 2,8 mm
kierroslukualue 15000-21000 min-1
Pituus 20 m
teho 1800 W
kapasiteetti 75,5 mm
kovapalaterä 210x30 mm 48 h
Pituus 90 mm
Korkeus noin 15 - 25 cm.
yhdyskaapeli 3 m (optiona 5/10 m)
saunan koko 7-12 m³
kivimäärä maks. 20 kg
Lujuus HRC 58
Kärki kolmiohiottu 4 mm.
Korkeus noin 10 cm.
kasvukorkeus n. 1-3 m
Naulat 18G 20-50 mm
Materiaali 65% polyesteriä, 35% puuvillaa , 300gr/m12
Pituus 50 mm
Pituus 75 

In [164]:
productean_to_attrs_ = {}
for ean, atrs in productean_to_attrs.items():
#     print(ean, ':', atrs)
    if len(atrs) != 0:
        productean_to_attrs_[ean] = atrs
productean_to_attrs = productean_to_attrs_

In [165]:
len(productean_to_attrs),len(productean_to_cat),len(productean_to_img),len(cat_to_producteans)

(879, 1005, 1005, 253)

In [166]:
for ean, atrs in list(productean_to_attrs.items())[:10]:
    print(ean, ':', atrs)

(500209664, ':', {u'kierrosluku': u'4800 min-1', u'ter\xe4': u'260x30, 48 hammasta', u'ottoteho': u'1650 W', u'kapasiteetti p\xf6yt\xe4': u'90\xb0  93 mm'})
(500230145, ':', {u'iskunvaimennuspohjallinen': u'C2', u'Pit\xe4v\xe4': u'grip-kumipohja'})
(500350635, ':', {u'puuvilla, twill': u'370g/m\xb2'})
(500642393, ':', {u'mukana': u'1 kpl hiomalaikka', u'\xba k\xe4\xe4ntyv\xe4 takakahva': u'180', u'mm laikka': u'230'})
(500342792, ':', {u'sementtipohjainen': u'jauhe', u'saumaleveyksille': u'2-6 mm', u'suihkutiivis EN': u'13888 CG2 WA', u'pitk\xe4': u'avoaika'})
(500350243, ':', {u'puuvilla, twill': u'370g/m\xb2'})
(500209676, ':', {u'Naulat': u'18G 20-50 mm'})
(500230158, ':', {u'iskunvaimennuspohjallinen': u'C2', u'Pit\xe4v\xe4': u'grip-kumipohja'})
(500068367, ':', {u'saatavissa': u'\xe4\xe4nieristysovia', u'vuoden tuotetakuu': u'2'})
(500068369, ':', {u'vuoden tuotetakuu': u'2', u'tehty': u'Suomessa'})


In [167]:
atr_count = {}
atr_val_count = {}
for ean, atrs in list(productean_to_attrs.items()):
    for atr, val in atrs.items():
        if atr not in atr_count:
            atr_count[atr] = 0
        atr_count[atr] += 1
        if (atr, val) not in atr_val_count:
            atr_val_count[(atr, val)] = 0
        atr_val_count[(atr, val)] += 1

In [168]:
for atr, count in sorted(atr_count.items(), key=lambda tup:tup[1], reverse=True)[:10]:
    print(atr, '_:_', count)

(u'Pituus', '_:_', 44)
(u'Korkeus noin', '_:_', 40)
(u'puuvilla, twill', '_:_', 38)
(u'Paino', '_:_', 37)
(u'ottoteho', '_:_', 35)
(u'kasvukorkeus n.', '_:_', 26)
(u'Materiaalit', '_:_', 24)
(u'teho', '_:_', 23)
(u'Tuotteen mitat', '_:_', 23)
(u'paino', '_:_', 21)


In [169]:
for atrval, count in sorted(atr_val_count.items(), key=lambda tup:tup[1], reverse=True)[:10]:
    print(atrval[0],'=',atrval[1], '_:_', count)

(u'puuvilla, twill', '=', u'370g/m\xb2', '_:_', 38)
(u'vuoden takuu', '=', u'10', '_:_', 17)
(u'2-komponentti', '=', u'kahvalla', '_:_', 16)
(u'100%', '=', u'polyesteri', '_:_', 15)
(u'3', '=', u'sauvainen', '_:_', 13)
(u'100 % puuvillatwill, 370 g/m\xb2', '=', u'100 % puuvillatwill, 370 g/m\xb2', '_:_', 13)
(u'vuoden tuotetakuu', '=', u'2', '_:_', 13)
(u'35% puuvilla twill, 240 g/m\xb2', '=', u'35% puuvilla twill, 240 g/m\xb2', '_:_', 11)
(u'65%', '=', u'polyesteri', '_:_', 11)
(u'polyesteri', '=', u'185g/m\xb2', '_:_', 10)
